In [3]:
import sys

sys.path.append("/Users/Mike/desktop/UCSB/CS/CS111/cs111-2021-fall/Python") 

import os
import time
import math
import numpy as np
import numpy.linalg as npla
import scipy
from scipy import linalg as spla
import scipy.sparse
import scipy.sparse.linalg
from scipy import integrate
import networkx as nx
import json
import cs111
import matplotlib
%matplotlib tk 
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d

np.set_printoptions(precision = 4)

## 1 - Temperature to Laplacian

In [34]:
def nonZeroRowSums(A):
    res = [sum(A[i,:]) for i in range(A.shape[0])]
    res = [i for i in res if i != 0]
    return res

In [43]:
A = cs111.make_A(100)
rows = nonZeroRowSums(A.toarray())
print("number of rows that need to be changed =", len(rows))

number of rows that need to be changed = 396


## 2.1 - How many cuts?

Given the formula $$xLx^T = \sum_{(i,j)\in E}{(x(i) - x(j))^2}$$ we can inspect what happens to the sum for every edge that crosses the cut. Now, the difference $$x(i) - x(j) = 0 \text{ or }2.$$ It is 0 if the $i$ and $j$ are on the same side of the cut, and it is 2 if the edge connecting $i$ and $j$ crosses the cut.$\newline\newline$ Let's say $i$ and $j$ have an edge between them. Then $$x(i) - x(j) = 2 \text{, and }x(j) - x(i) = 2.$$ This $2$ is then squared through the $$(x(i) - x(j))^2$$ expression. As a result, the edge the crosses the cut produces a value of $8$ through the summation instead of the necessary value of $2$.$\newline$This means that each cut is counted $\frac{8}{2}=4$ times. Since $$xLx^T = \sum_{(i,j)\in E}{(x(i) - x(j))^2},$$ we can conclude that the number of edges crossing a cut in any graph is $$\frac{xLx^T}{4},$$ meaning that our value $\alpha$ is $$\alpha = \frac{1}{4}$$

## 2.2 - Coordinate Cut

In [4]:
# load the airfoil graph
G, xycoords = cs111.read_mesh('airfoil1')
# calculate the median
new_xy = [xycoords[i] for i in range(len(xycoords))]
median_coordinate = np.median(new_xy, axis = 0)[1] # median of the y coordinate

In [47]:
# plot
first_colors = ['r' if xycoords[v][1] < median_coordinate else 'b' for v in G.nodes()]
plt.ion()
plt.figure(figsize=(6,6))
plt.title('geometric partition')
plt.axis('equal')
nx.draw(G, pos = xycoords, node_size = 4, node_shape = 'o', node_color = first_colors)

In [45]:
# calculate vector x based on the red/blue coloring
my_list = [1 if first_colors[i] == 'r' else -1 for i in range(len(first_colors))]
x = np.array(my_list)
# make the laplacian of G
L = nx.linalg.laplacian_matrix(G).toarray()
print("num of edges that cross the cut = ", x @ L @ x.T * 1/4)
print("num of red points =" , first_colors.count('r'))
print("num of blue points =" , first_colors.count('b'))

num of edges that cross the cut =  148.0
num of red points = 2126
num of blue points = 2127


In [49]:
plt.close('all')

## 2.3 - Spectral Cut

In [9]:
# make the laplacian of G
L = nx.linalg.laplacian_matrix(G).toarray()
# eigenvectors of the laplacian L
lam, W = spla.eigh(L)
# finding the fiedler vector of L
fiedler_vector = np.transpose(W)[1]
# calculate the median of fiedler_vector
median_fiedler_vec_value = np.median(fiedler_vector)

In [48]:
# now we label each element with the corresponding element
# of the Fiedler vector and color it based on whether that 
# label is smaller or larger than the median value we just computed
second_colors = ['r' if fiedler_vector[v] < median_fiedler_vec_value else 'b' for v in G.nodes()]
plt.ion()
plt.figure(figsize=(6,6))
plt.title('spectral partition')
plt.axis('equal')
nx.draw(G, pos = xycoords, node_size = 4, node_shape = 'o', node_color = second_colors)

In [46]:
# calculate vector x based on the red/blue coloring
my_list = [1 if second_colors[i] == 'r' else -1 for i in range(len(second_colors))]
x = np.array(my_list)
# how many edges cross the cut?
print("num of edges that cross the cut =", x @ L @ x.T * 1/4)
print("num of red points =", my_list.count(1))
print("num of blue points =", my_list.count(-1))

num of edges that cross the cut = 132.0
num of red points = 2126
num of blue points = 2127
